# Data Preprocressing - Aspect Extraction

In [ ]:
# pip install fasttext

In [ ]:
# pip install stanfordnlp

In [1]:
import stanfordnlp
import pandas as pd
import numpy as np
import nltk

In [3]:
df = pd.read_csv('../datasets/reviews.csv')

In [4]:
df.head()

,Unnamed: 0,title,body,rating,product
0,0,Great wireless music,Product as expecred,5.0,Apple Airpods-Latest Model
1,1,battery life dies really quick,i really like my airpods but the battery life ...,4.0,Apple Airpods-Latest Model
2,2,The audio sound muffled (right ear),I bought Airpods 2 on Amazon. But right ears w...,3.0,Apple Airpods-Latest Model
3,3,Comprarlo sin problemas,Perfect,5.0,Apple Airpods-Latest Model
4,4,good！,so,5.0,Apple Airpods-Latest Model


In [5]:
# Using RegEx to 
df['title'] = df['title'].apply(lambda x: x.lower())
df['body'] = df['body'].apply(lambda x: x.lower())
# remove punctuation from text
df['title'] = df['title'].str.replace('[^\w\s]','')
df['body'] = df['body'].str.replace('[^\w\s]','')

In [6]:
df.head()

,Unnamed: 0,title,body,rating,product
0,0,great wireless music,product as expecred,5.0,Apple Airpods-Latest Model
1,1,battery life dies really quick,i really like my airpods but the battery life ...,4.0,Apple Airpods-Latest Model
2,2,the audio sound muffled right ear,i bought airpods 2 on amazon but right ears wa...,3.0,Apple Airpods-Latest Model
3,3,comprarlo sin problemas,perfect,5.0,Apple Airpods-Latest Model
4,4,good,so,5.0,Apple Airpods-Latest Model


In [7]:
# def start(): 
  
#     # empty list to store the contents of  
#     # the website fetched from our web-crawler 
#     wordlist = [] 
# #     source_code = requests.get(url).text   
  
#     # Text in given web-page is stored under 
#     # the <div> tags with class <entry-content> 
#     for each_text in soup.findAll('div', {'class':'entry-content'}): 
#         content = each_text.text 
  
#         # use split() to break the sentence into  
#         # words and convert them into lowercase  
#         words = content.lower().split() 
          
#         for each_word in words: 
#             wordlist.append(each_word) 
#         clean_wordlist(wordlist) 
  
# # Function removes any unwanted symbols 
# def clean_wordlist(wordlist): 
      
#     clean_list =[] 
#     for word in wordlist: 
#         symbols = '!@#$%^&*()_-+={[}]|\;:"<>?/., '
          
#         for i in range (0, len(symbols)): 
#             word = word.replace(symbols[i], '') 
              
#         if len(word) > 0: 
#             clean_list.append(word) 
#     create_dictionary(clean_list) 
  
# # Creates a dictionary conatining each word's  
# # count and top_20 ocuuring words 
# def create_dictionary(clean_list): 
#     word_count = {} 
      
#     for word in clean_list: 
#         if word in word_count: 
#             word_count[word] += 1
#         else: 
#             word_count[word] = 1
              
#     ''' To get count of each word in 
#         the crawled page --> 
          
#     # operator.itemgetter() takes one  
#     # parameter either 1(denotes keys) 
#     # or 0 (denotes corresponding values) 
      
#     for key, value in sorted(word_count.items(), 
#                     key = operator.itemgetter(1)): 
#         print ("% s : % s " % (key, value)) 
          
#     <-- '''
  
      
#     c = Counter(word_count) 
      
#     # returns the most occurring elements 
#     top = c.most_common(10) 
#     print(top)

In [9]:
from collections import Counter

In [11]:
# Creating a dictionary of the most common words
word_dict = Counter(" ".join(df['body']).split(' ')).items()

In [18]:
counter = pd.DataFrame(word_dict, columns=['word', 'count'])
counter.head()

,word,count
0,product,7463
1,as,7370
2,expecred,1
3,i,42816
4,really,3455


In [20]:
counter.groupby('count').max()

,word
count,
1,सगत
2,ıts
3,ı
4,único
5,yea
...,...
27871,is
33453,to
39051,and


In [27]:
counter.loc[counter['word']=='battery']

,word,count
10,battery,4474


In [28]:
counter.loc[counter['word']=='cancellation']

,word,count
685,cancellation,1220


In [30]:
counter.loc[counter['word']=='quality']

,word,count
59,quality,11077


In [31]:
counter.loc[counter['word']=='price']

,word,count
341,price,3566


In [32]:
counter.loc[counter['word']=='noise']

,word,count
515,noise,2761


In [33]:
counter.loc[counter['word']=='fit']

,word,count
683,fit,2919


In [34]:
counter.loc[counter['word']=='comfort']

,word,count
1932,comfort,268


In [35]:
counter.loc[counter['word']=='comfortable']

,word,count
414,comfortable,1391


In [36]:
counter.loc[counter['word']=='not']

,word,count
68,not,12299


In [37]:
# From a dash of data - making sure the data is fully cleaned
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [38]:
df_clean = pd.DataFrame(df.body.apply(round1))

In [39]:
df_clean.head()

,body
0,product as expecred
1,i really like my airpods but the battery life ...
2,i bought airpods on amazon but right ears was...
3,perfect
4,so


In [40]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [41]:
df_clean = pd.DataFrame(df_clean.body.apply(round2))

In [43]:
# Saving cleaned dataframe
df_clean.to_csv('../datasets/df_clean.csv')

In [44]:
# Creating a document term matrix using CountVectorizer and eliminating 
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
df_cv = cv.fit_transform(df_clean.body)
df_dtm = pd.DataFrame(df_cv.toarray(), columns=cv.get_feature_names())
df_dtm.index = df_clean.index
df_dtm

,aa,aaa,aaaaalmost,aaaandthe,aac,aacept,aata,aavaz,aay,ab,...,клавиш,микрофон,наушники,не,отличный,сvс,супер,только,хватает,सगत
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31863,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31864,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31865,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31866,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Saving the DTM to a new CSV, potentially to be used later
df_dtm.to_csv('../datasets/df_dtm.csv')